In [ ]:
import sympy as sp

# Defina as variáveis simbólicas
deltaL, Lo, deltaT, alpha = sp.symbols('deltaL Lo deltaT alpha')

# Defina a expressão para alpha
alpha_expr = deltaL / (Lo * deltaT)

# Calcule a primeira derivada parcial de alpha em relação a DeltaL
derivada_parcial_deltaL = sp.diff(alpha_expr, deltaL)

# Exiba o resultado
print(derivada_parcial_deltaL)


1/(Lo*deltaT)


**Testando aumotizar os cálculos direto do excel**

In [ ]:

import sympy as sp
import pandas as pd


dados = pd.read_excel('dados.xlsx')

# Defina as variáveis simbólicas
x, y, z = sp.symbols('x y z')

# Defina a expressão
expr = x**2 + y**2 + z**2


# Obtenha os valores do array para substituição
coluna1 = dados['x'].values
print(coluna1)
y_valor = dados['y'].values
print(y_valor)

# Substitua os valores diretamente na expressão
expr_substituida = expr.subs({x: coluna1[0], y: y_valor[0], z: 4})
print("fazendo direto pelo excel",expr_substituida)

FileNotFoundError: ignored

**Vou aplicar isso para as derivadas**

**Deu certo a 1º tentativa para deltaL, agora irei calcular para as demais derivadas**

In [6]:
import pandas as pd
import sympy as sp

# Carregue o arquivo Excel
dados2 = pd.read_excel('dados2.xlsx')

# Transforme os dados em um dicionário
dados_dict = dict(zip(dados2['variavel'], dados2['valor']))

# Exiba o dicionário
print("dicionarios dos dados em excel", dados_dict)


# Defina as variáveis simbólicas
deltaL, Lo, deltaT, alpha = sp.symbols('deltaL Lo deltaT alpha')

# Defina a expressão para alpha
alpha_expr = deltaL / (Lo * deltaT)

# Calcule a primeira derivada parcial de alpha em relação a DeltaL
derivada_deltaL = sp.diff(alpha_expr, deltaL)
print("derivada parcial para deltaL:", derivada_deltaL)

# Avalie a derivada no ponto específico
valor_da_derivada_em_ponto = derivada_deltaL.subs(dados_dict)

# Exiba o resultado
print("calculando a derivada com os dados em excel",valor_da_derivada_em_ponto)

dicionarios dos dados em excel {'deltaL': 0.29, 'Lo': 22.0, 'deltaT': 25.0, 'alpha': 0.00027}
derivada parcial para deltaL: 1/(Lo*deltaT)
calculando a derivada com os dados em excel 0.00181818181818182


**Derivada parcial para Lo**

In [7]:

# Defina a expressão para alpha
alpha_expr = deltaL / (Lo * deltaT)

# Calcule a primeira derivada parcial de alpha em relação a DeltaL
derivada_Lo = sp.diff(alpha_expr, Lo)
print("derivada parcial para deltaL:", derivada_Lo)

# Avalie a derivada no ponto específico
valor_da_derivada_em_ponto = derivada_Lo.subs(dados_dict)

# Exiba o resultado
print("calculando a derivada com os dados em excel",valor_da_derivada_em_ponto)

derivada parcial para deltaL: -deltaL/(Lo**2*deltaT)
calculando a derivada com os dados em excel -2.39669421487603e-5
